In [1]:
import os
import errno
import datetime
import time
from zipfile import ZipFile
import json
from pathlib import Path
import tomli
import geopandas as gpd
import shapely
import requests
import rasterio as rio
from requests.auth import HTTPBasicAuth
import matplotlib.pyplot as plt
import numpy as np
import glob
import aiohttp
import asyncio
# from geojson import Polygon, Feature, FeatureCollection

In [4]:
# sievers_bound = gpd.read_file('data/sievers_bound.geojson')
# cas_bound = gpd.read_file('data/sievers_bound.geojson')
# accola_bound = gpd.read_file('data/sievers_bound.geojson')
# mud_bound = gpd.read_file('data/mud_creek_watershed_4326.geojson')

In [17]:
# from distutils.command.config import config
class PlanetConfig:
    def __init__(self, config_file="config.toml"):
        self.config_file = config_file
        # self.order_url = 'https://api.planet.com/compute/ops/orders/v2'
        self.headers = {'content-type': 'application/json'}
        
        try:
            with open(config_file, "rb") as f:
                self.config = tomli.load(f)
        except EnvironmentError as e:
            print(os.strerror(e.errno))
            print("Missing configiguration file.")
            print("Please create a config.toml file with your Planet API key.")
            print("Use config.toml.example as a template.")
        match self.config:
            case {
                "api": {"planet_api_key": str(), 'item_type': str(), 'image_type': str()},
                "filters": {"mask": str(), 'max_cloud': float(), 'start_date': str(), 'end_date': str()},
            }:
                pass
            case ValueError as e:
                print(f'Missing or incorrect value in config.toml')
                print(str(e))
        self.API_KEY = self.config['api']['planet_api_key']
        self.SEARCH_URL = 'https://api.planet.com/data/v1/quick-search'
        self.ITEM_TYPE = self.config['api']['item_type']
        self.IMAGE_TYPE = self.config['api']['image_type']
        self.project_name = self.config['general']['project_name']
        self.auth = HTTPBasicAuth(self.API_KEY, '')
        self.mask = self.config['filters']['mask']
        self.max_cloud = self.config['filters']['max_cloud']
        self.start_date = self.config['filters']['start_date']
        self.end_date = self.config['filters']['end_date']
        
    def __repr__(self) -> str:
        return (
            f"Configuration and filters for Planet API image aquisition"
        )
    
        
    

In [3]:
my_config = PlanetConfig()
my_config.API_KEY

'PLAK7f862c713f3243fb81cd6b6ce6e26f45'

In [69]:
class PlanetImages:
    def __init__(self):
        self.config = PlanetConfig()
        self.date_folder = f'{self.config.start_date}_{self.config.end_date}'
        self.thumb_dir = os.path.join('data', 'imagery', f'{self.config.project_name}', f'{self.date_folder}', 'thumbnails')
        self.img_dir = os.path.join('data', 'imagery', f'{self.config.project_name}', f'{self.date_folder}')
        self.search_json = {}
        self.img_count = 0
        self.image_ids = []
        self.image_list = []
        self.unique_dates = []
        self.good_imgs = []
        self.imgs_to_download = []
        self.active_imgs = []
        self.all_imgs_active = False
        self.downloaded_imgs = []
    
    def _load_aoi(self):
        aoi_geom = gpd.read_file(self.config.mask)
        aoi_geom = aoi_geom.to_json()
        aoi_geom = json.loads(aoi_geom)
        aoi_coords = aoi_geom['features'][0]['geometry']
        return aoi_coords
    
    def search_for_images(self):
        aoi = self._load_aoi()
        #set start and end dates
        start_date = self.config.start_date + 'T00:00:00.000Z'
        end_date = self.config.end_date + 'T00:00:00.000Z'
        #set the mask or filter for desired AOI
        geometry_filter = {
        "type": "GeometryFilter",
        "field_name": "geometry",
        "config": aoi
        }
        #set range of dates to search for
        date_range_filter = {
        "type": "DateRangeFilter",
        "field_name": "acquired",
        "config": {
            "gte": start_date,
            "lte": end_date
        }
        }

        # filter any images which are more than 10% clouds
        cloud_cover_filter = {
        "type": "RangeFilter",
        "field_name": "cloud_cover",
        "config": {
            "lte": self.config.max_cloud
        }
        }

        # create a filter that combines our geo and date filters
        # could also use an "OrFilter"
        combined_filter = {
        "type": "AndFilter",
        "config": [geometry_filter, date_range_filter, cloud_cover_filter]
        }
        #set what type of Planet scene we want to download
        # API request object
        search_request = {
        "item_types": [self.config.ITEM_TYPE], 
        "filter": combined_filter
        }
        search_result = requests.post(
            self.config.SEARCH_URL,
            auth=HTTPBasicAuth(self.config.API_KEY, ''), json=search_request)
        self.search_json = search_result.json()
        return self.search_json
    
    def get_all_avail_image_ids(self):
        #just return which images are available
        image_json = self.search_for_images()
        self.image_ids = [feature['id'] for feature in image_json['features']]
        print(f'There are {len(self.image_ids)} total images available to download.')
        return self.image_ids
    
    def get_unique_image_dates(self) -> list:
        image_json = self.search_for_images()
        self.image_list = []
        self.unique_dates = []
        #loop through all images, get unique date, and return image info/feature info
        for feature in image_json['features']:
            image_id = feature['id']
            date = image_id[0:8]
            if date not in self.unique_dates:
                self.unique_dates.append(date)
                self.image_list.append(feature)
        num_images = len(self.image_list)
        if num_images != 0:
            print(f'There are {num_images} unique image dates for download.')
            return self.image_list
        else:
            raise ValueError("Image list is empty. Try a new date, location, or filters.")

    def download_image_thumbnails(self) -> None:
        image_list = self.image_list
        img_total = len(image_list)
        counter = 0
        if not os.path.exists(self.thumb_dir):
            os.makedirs(self.thumb_dir)
        for image in image_list:
            thumb_url = image['_links']['thumbnail']
            img_id = image['id']
            thumb_req = requests.get(thumb_url, auth=HTTPBasicAuth(self.config.API_KEY, ''))
            downloaded_thumb = glob.glob(f'{self.thumb_dir}/{img_id}.tif')
            if not downloaded_thumb:
                # print(f"Downloading thumbnail for {img_id}")
                open(f'{self.thumb_dir}/{img_id}.tif', 'wb').write(thumb_req.content)
            else:
                pass
            # print(feature['_links']['thumbnail'])
            # counter += 1
        # return(image_list)

    def filter_images_for_quality(self) -> list:
        thumb_imgs = glob.glob(f'{self.thumb_dir}/*.tif')
        self.good_imgs = []
        for image in thumb_imgs:
            with rio.open(image) as src:
                b, g, r, nir = src.read()
                b_mean = b.mean()
                g_mean = g.mean()
                r_mean = r.mean()
                nir_mean = nir.mean()
                # print(image, b_mean, g_mean, r_mean, nir_mean)
                if nir_mean >= 100 and r_mean < 10:
                    print(f'{image} is possibly bad and/or corrupted. Double check before downloading.')
                else:
                    self.good_imgs.append(image)
        return self.good_imgs

    def get_imgs_to_download(self):
        img_base_names = [os.path.basename(img) for img in self.good_imgs]
        img_base_names = [i[:len(i)-4] for i in img_base_names]
        self.imgs_to_download = [i for i in self.search_json['features'] if i['id'] in img_base_names]
        return self.imgs_to_download

    def activate_imgs(self):
        for i in self.imgs_to_download:
            img_id = i['id']
            img_links = i['_links']
            # dwn_link = img_links['_self']
            asset_url = f'https://api.planet.com/data/v1/item-types/{my_config.ITEM_TYPE}/items/{img_id}/assets'
            try:
                result = \
                requests.get(
                    asset_url,
                    auth=HTTPBasicAuth(my_config.API_KEY, '')
                )
                img_status = result.json()[f'{my_config.IMAGE_TYPE}']['status']
                if img_status == 'inactive':
                    print(f'Image {img_id} is inactive. Attempting to activate.')
                    links = result.json()[f'{my_config.IMAGE_TYPE}']["_links"]
                    # self_link = links["_self"]
                    activation_link = links["activate"]
                    # Request activation of the 'analytic' asset:
                    activate_result = \
                    requests.get(
                        activation_link,
                        auth=HTTPBasicAuth(my_config.API_KEY, '')
                    )
                elif img_status == 'active':
                    print(f'Image {img_id} is already active.')
                else:
                    print(f'Unknown status for image {img_id}')
                    self.imgs_to_download.remove(i)
            except KeyError:
                print(f'Image type {my_config.IMAGE_TYPE} not available for {img_id}')
                print(f'Removing image {img_id} from download list.')
                print(f'These other images are available for {img_id}: {result.json().keys()}')
                self.imgs_to_download.remove(i)
        
    def check_if_images_active(self):
        # check to see if product is active or not
        num_loops = 21
        count = 0
        while(count < num_loops):
            for i in self.imgs_to_download:
                img_id = i['id']
                img_links = i['_links']
                asset_url = img_links['assets']
                # self_link = img_links['_self']
                result = \
                    requests.get(
                        asset_url,
                        auth=HTTPBasicAuth(my_config.API_KEY, '')
                    )
                img_status = result.json()[f'{my_config.IMAGE_TYPE}']['status']   
                success_states = ['active']
                if img_status == 'failed':
                    raise Exception()
                elif img_status in success_states and i not in self.active_imgs:
                    self.active_imgs.append(i)
            count += 1
            if len(self.active_imgs) == len(self.imgs_to_download):
                print('All images ready to download.')
                self.all_imgs_active = True
                break
            time.sleep(30)

    def download_images(self):
        if self.all_imgs_active == True:
            print('Downloading images.')
        else:
            print('All images may not be active. Downloading those that are.')
            print('You may want to run this script again after.')
        ###TODO repeating a lot of requests. Clean this up later and implement other requests above.
        ###TODO mainly there are 2 '_self' links and both are needed. The first is needed to get download link.
        for image in self.active_imgs:
            #go through all active images
            img_id = image['id']
            asset_url = f'https://api.planet.com/data/v1/item-types/{my_config.ITEM_TYPE}/items/{img_id}/assets'
            try:
                #get the asset info
                result = \
                requests.get(
                    asset_url,
                    auth=HTTPBasicAuth(my_config.API_KEY, '')
                )
                #get different _self link for these assets
                links = result.json()[f'{my_config.IMAGE_TYPE}']['_links']
                self_link = links['_self']
                self_req = \
                    requests.get(
                    self_link,
                    auth=HTTPBasicAuth(my_config.API_KEY, '')
                )
                download_url = self_req.json()["location"]
                img_req = requests.get(download_url, auth=HTTPBasicAuth(self.config.API_KEY, ''))
                downloaded_img = glob.glob(f'{self.img_dir}/{img_id}.tif')
                if not downloaded_img:
                    print(f"Downloading image {img_id}.")
                    open(f'{self.img_dir}/{img_id}.tif', 'wb').write(img_req.content)
                else:
                    print(f'Image {img_id} already downloaded -- skipping.')
                    pass
            except:
                print('Something went wrong.')
    
        

In [70]:
plan_imgs = PlanetImages()
plan_imgs.get_all_avail_image_ids()
plan_imgs.get_unique_image_dates()
plan_imgs.download_image_thumbnails()
plan_imgs.filter_images_for_quality()
plan_imgs.get_imgs_to_download()
plan_imgs.activate_imgs()
plan_imgs.check_if_images_active()
plan_imgs.download_images()

There are 31 total images available to download.
There are 17 unique image dates for download.
data\imagery\Sievers\2019-07-01_2019-07-31\thumbnails\20190708_164141_67_1059.tif is possibly bad and/or corrupted. Double check before downloading.


c:\Users\mattn\miniconda3\envs\gis\Lib\site-packages\rasterio\__init__.py:321: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Image 20190730_163301_101b is already active.
Image 20190728_162818_1001 is already active.
Image 20190725_151708_1054 is already active.
Image 20190724_151544_0f2b is already active.
Image 20190723_151601_1020 is already active.
Image 20190722_162912_101b is already active.
Image 20190719_163108_0f3f is already active.
Image 20190718_165000_14_1067 is already active.
Image 20190714_163108_0f17 is already active.
Image 20190709_162920_1013 is already active.
Image 20190707_151825_0f4d is already active.
Image 20190705_163123_1006 is already active.
Image 20190704_163219_0f4e is already active.
Image 20190703_152135_0f49 is already active.
Image 20190702_151809_1020 is already active.
Image type ortho_analytic_4b_sr not available for 20190701_204722_0f1c
Removing image 20190701_204722_0f1c from download list.
These other images are available for 20190701_204722_0f1c: dict_keys(['basic_udm2', 'ortho_udm2', 'ortho_visual'])
All images ready to download.


In [ ]:
# # create an api request from the search specifications
# def build_request(aoi_geom, start_date, stop_date):
#     '''build a data api search request for clear PSScene 4-Band imagery'''
#     item_type = 'PSScene'
#     query = filters.and_filter(
#         filters.geom_filter(aoi_geom),
#         filters.range_filter('clear_percent', gte=90),
#         filters.date_range('acquired', gt=start_date),
#         filters.date_range('acquired', lt=stop_date)
#     )
#     return filters.build_search_request(query, ['PSScene'])

In [ ]:
request = build_request(aoi_coords, start_date, stop_date)
request

In [ ]:
# search the data api
def search_data_api(request, client, limit=500):
    result = client.quick_search(request)
    
    # this returns a generator
    return result.items_iter(limit=limit)

items = list(search_data_api(request, client))
print(len(items))

In [ ]:
test_items = items[:2]
# filter to item ids
ids = [i['id'] for i in test_items]
ids

In [ ]:
name = 'tutorial_order'
item_type = 'PSScene'
# 'analytic_sr_udm2' is the basic surface reflectance corrected image, 4 bands
bundle = 'analytic_sr_udm2'
# and the 8 band version
# bundle = 'analytic_8b_sr_udm2'
clip_tool = {'clip': {'aoi': aoi_coords}}
# # example of a bandmath tool to calculate NDVI that will overwrite default bands
# bandmath_tool = {'bandmath': {
#     "pixel_type": "32R",
#     "b1": "(b4 - b3) / (b4+b3)",
#     "b2": "(b4 / b2) - 1",
# }}

# tools = [clip_tool, bandmath_tool]
tools = clip_tool

orders_request = {
    'name': name,
    'products': [{
        'item_ids': ids,
        'item_type': item_type,
        'product_bundle': bundle
    }],
    'tools': tools,
    'delivery': {
        'single_archive': True,
        'archive_filename':'{{name}}_{{order_id}}.zip',
        'archive_type':'zip'
    },
        'notifications': {
                   'email': False
    },
}

# pprint(orders_request, indent=1)

In [ ]:
order_info = client.create_order(orders_request).get()

order_id = order_info['id']
order_id

### Edit below to check if asset is ready and then download

In [ ]:
order_info['_links']['_self']

In [ ]:
def poll_for_success(order_id, client, num_loops=50) -> None:
    count = 0
    while(count < num_loops):
        count += 1
        order_info = client.get_individual_order(order_id).get()
        state = order_info['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(30)
        
poll_for_success(order_id, client)

In [ ]:
demo_data_dir = os.path.join('data', 'demo')
# make the download directory if it doesn't exist
Path(demo_data_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
orders_url = order_info['_links']['_self']

In [ ]:
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join('data', n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))

In [ ]:
%system planet orders download 4fc4e8f9-58e4-4521-966d-56a34ee41797